# gestion des secrets

[source](https://savelev.medium.com/best-practice-how-to-store-secrets-and-settings-in-python-project-e3ee45b3094c)

In [1]:
import os
import json
def read_secrets() -> dict:
    filename = os.path.join('secrets.json')
    try:
        with open(filename, mode='r') as f:
            return json.loads(f.read())
    except FileNotFoundError:
        return {}
secrets = read_secrets()

# test accès stripe

In [2]:
# import stripe
# stripe.api_key = secrets['stripe.api_key_test']

# # list customers
# customers = stripe.Customer.list()

# # print the first customer's email
# print(customers.data[0].email)

# # retrieve specific Customer
# # customer = stripe.Customer.retrieve("cus_123456789")

# # print that customer's email
# # print(customer.email)

# exeuction rapport

In [3]:
# from tqdm import tqdm
# import time

# def long_running_function(n, timeout=5):
#     start_time = time.time()

#     for _ in tqdm(list(range(n))):
#         time.sleep(1)  # doing some expensive work...
#         elapsed_time = time.time() - start_time
#         if elapsed_time > timeout:
#             raise TimeoutError("long_running_function took too long!")


# long_running_function(100, timeout=10)

  9%|▉         | 9/100 [00:10<01:41,  1.11s/it]


TimeoutError: long_running_function took too long!

In [25]:
from tqdm import tqdm
for i in tqdm(range(10000000)):
    pass

100%|██████████| 10000000/10000000 [00:01<00:00, 6958596.86it/s]


## gestion date début/fin rapport

[lien vers doc de stripe](https://stripe.com/docs/api/reporting/report_run/object)

[différents type de rapport stripe](https://stripe.com/docs/reports/report-types)

In [63]:
import stripe
import requests
import pandas as pd
import io
from woocommerce import API
from datetime import datetime
import time

stripe.api_key = secrets['stripe.api_key']

def date_converter(date):
  """convert date to timestamp and isoformat

  Args:
      date (exemple 20/01/22)

  Returns:
      _type_: renvoi la date en timestamp & la date en isoformat
  """
  date_timestamp = int(time.mktime(datetime.strptime(date, "%d/%m/%Y").timetuple()))
  date_isoformat = datetime.fromtimestamp(date_timestamp).isoformat()
  return date_timestamp, date_isoformat

def convert_to_isoformat(date_timestamp):
	return datetime.fromtimestamp(date_timestamp).isoformat()

def get_stripe_data(start_date, end_date, PAUSE=10):
  report = stripe.reporting.ReportRun.create(
    report_type="payout_reconciliation.itemized.5",
    parameters={
      "interval_start": start_date,
      "interval_end": end_date
      # "columns": ['automatic_payout_id', 
      # 'automatic_payout_effective_at',
      # 'balance_transaction_id',
      # 'created', 
      # 'available_on',
      # 'currency',
      # 'gross',
      # 'fee',
      # 'net',
      # 'reporting_category',
      # # 'source_id',
      # 'description'
      # 'customer_id',
      # 'customer_email',
      # 'customer_name',
      # 'customer_description',
      # 'invoice_id',
      # 'order_id',
      # 'payment_method_type'
      # ]
    },
  )


  timeout = time.time() + 60*20   # 20 minutes from now
  i=0

  id = report['id']
  print(id)

  while time.time() < timeout:
      report = stripe.reporting.ReportRun.retrieve(id)
      if report['status'] == 'succeeded':
          print(f"Execution {i+1} +{PAUSE*i}sec : {report['status']}")
          r = requests.get(report['result']['url'], auth=(stripe.api_key,""))
          urlData = r.content
          df = pd.read_csv(io.StringIO(urlData.decode('utf-8')))
          print('stripe_data_ok')
          break
      else :
          print(f"Execution {i+1} +{PAUSE*i}sec : {report['status']}")
      i +=1
      time.sleep(PAUSE)
  return df


def get_woocommerce_data(start_date_isoformat, end_date_isoformat):
	#init api
  wcapi = API(
    url=secrets['woocommerce.url'], # Your store URL
    consumer_key=secrets['woocommerce.key'], # Your consumer key
    consumer_secret=secrets['woocommerce.secret'], # Your consumer secret
    wp_api=True, # Enable the WP REST API integration
    version="wc/v3" # WooCommerce WP REST API version
  )
  nb_commandes = wcapi.get("orders")
  nb_commandes = nb_commandes.headers['X-WP-Total']
  commandes = wcapi.get(f"orders?per_page=30&after={start_date_isoformat}&before={end_date_isoformat}")
  df_commandes = pd.DataFrame(commandes.json())
  df_commandes = df_commandes[[
	"id",
	"status",
	"currency",
	"date_created",
	"date_modified",
	"total",
	"payment_method",
	"customer_note",
	"line_items",
	"refunds"
  ]]
  print('woocommerce_data_ok')
  return df_commandes   

In [32]:
start_date = "01/10/2022"
end_date = "31/10/2022"
start_date_timestamp, start_date_isoformat = date_converter(start_date)
end_date_timestamp, end_date_isoformat = date_converter(end_date)
report = stripe.reporting.ReportRun.create(
report_type="payout_reconciliation.itemized.5",
parameters={
"interval_start": start_date_timestamp,
"interval_end": end_date_timestamp
# "columns": ['automatic_payout_id', 
# 'automatic_payout_effective_at',
# 'balance_transaction_id',
# 'created', 
# 'available_on',
# 'currency',
# 'gross',
# 'fee',
# 'net',
# 'reporting_category',
# # 'source_id',
# 'description'
# 'customer_id',
# 'customer_email',
# 'customer_name',
# 'customer_description',
# 'invoice_id',
# 'order_id',
# 'payment_method_type'
# ]
},
)

In [39]:
report = stripe.reporting.ReportRun.retrieve("frr_1LzM7CJeZLXFAOnZVx03FtN3")
report

<ReportRun reporting.report_run id=frr_1LzM7CJeZLXFAOnZVx03FtN3 at 0x7fa07b64fc40> JSON: {
  "created": 1667315458,
  "error": null,
  "id": "frr_1LzM7CJeZLXFAOnZVx03FtN3",
  "livemode": true,
  "object": "reporting.report_run",
  "parameters": {
    "interval_end": 1667170800,
    "interval_start": 1664575200
  },
  "report_type": "payout_reconciliation.itemized.5",
  "result": null,
  "status": "pending",
  "succeeded_at": null
}

In [64]:
PAUSE = 10
start_date = "01/10/2022"
end_date = "01/11/2022"
start_date_timestamp, start_date_isoformat = date_converter(start_date)
end_date_timestamp, end_date_isoformat = date_converter(end_date)

df_stripe = get_stripe_data(start_date_timestamp, end_date_timestamp,PAUSE)
df_woocommerce = get_woocommerce_data(start_date_isoformat, end_date_isoformat)


frr_1LzSMNJeZLXFAOnZTKk7niNI
Execution 1 +0sec : succeeded
stripe_data_ok
woocommerce_data_ok


In [31]:
report

NameError: name 'report' is not defined

In [65]:
df_stripe.head()

,automatic_payout_id,automatic_payout_effective_at,balance_transaction_id,created,available_on,currency,gross,fee,net,reporting_category,description
0,po_1LxLgfJeZLXFAOnZxNZH9xJZ,2022-10-27 02:19:17,txn_3Lw733JeZLXFAOnZ0lqUL9pm,2022-10-23 16:29:18,2022-10-27 00:00:00,eur,36.0,0.75,35.25,charge,APE Dracy-le-Fort - Commande 453
1,po_1LxLgfJeZLXFAOnZxNZH9xJZ,2022-10-27 02:19:17,txn_3LwPqoJeZLXFAOnZ1F3AVSz5,2022-10-24 12:33:55,2022-10-27 00:00:00,eur,30.0,0.67,29.33,charge,APE Dracy-le-Fort - Commande 457
2,po_1LxLgfJeZLXFAOnZxNZH9xJZ,2022-10-27 02:19:17,txn_3LwVXNJeZLXFAOnZ091wsAPG,2022-10-24 18:38:14,2022-10-27 00:00:00,eur,36.0,0.75,35.25,charge,APE Dracy-le-Fort - Commande 458
3,po_1LynI1JeZLXFAOnZjxRikPwd,2022-10-31 01:59:49,txn_3Lx4KxJeZLXFAOnZ1vJGWLNk,2022-10-26 07:47:44,2022-10-31 00:00:00,eur,31.0,0.68,30.32,charge,APE Dracy-le-Fort - Commande 459
4,po_1LynI1JeZLXFAOnZjxRikPwd,2022-10-31 01:59:49,txn_3LxBVLJeZLXFAOnZ1VzNKKZz,2022-10-26 15:26:56,2022-10-31 00:00:00,eur,64.0,1.15,62.85,charge,APE Dracy-le-Fort - Commande 460


In [66]:
df_stripe['id'] = [int(x.split(' ')[-1]) for x in df_stripe['description']]
df_stripe[['description', 'id']]

,description,id
0,APE Dracy-le-Fort - Commande 453,453
1,APE Dracy-le-Fort - Commande 457,457
2,APE Dracy-le-Fort - Commande 458,458
3,APE Dracy-le-Fort - Commande 459,459
4,APE Dracy-le-Fort - Commande 460,460


In [67]:
df_merged = df_woocommerce.merge(df_stripe, on='id', how='outer')
df_merged.to_csv('data.csv')

In [89]:
test = df_merged.explode('line_items',ignore_index=True)
# test['line_items'].to_list()
test[['id', 'name', 'product_id', 'variation_id', 'quantity', 'tax_class',
       'subtotal', 'subtotal_tax', 'total', 'total_tax', 'taxes', 'meta_data',
       'sku', 'price', 'image', 'parent_name']] = pd.DataFrame(test['line_items'].to_list(), index= test.index)